In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [33]:
!pip uninstall pandas -y

Found existing installation: pandas 0.24.2
Uninstalling pandas-0.24.2:
  Successfully uninstalled pandas-0.24.2


In [34]:
!pip install pandas

     |████████████████████████████████| 9.9 MB 7.5 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 2.8.0 requires requests>=2.23.0, but you have requests 2.21.0 which is incompatible.
osmnx 1.0.1 requires requests>=2.25, but you have requests 2.21.0 which is incompatible.
google-colab 1.0.0 requires pandas~=0.24.0, but you have pandas 1.2.3 which is incompatible.
autogluon-core 0.1.0b20210219 requires tornado>=5.0.1, but you have tornado 4.5.3 which is incompatible.


In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('/kaggle/input/unsafe/train_randst0.csv')
test = pd.read_csv('/kaggle/input/unsafe/val_randst0.csv')
train = train[["text", "unsafe"]]
test = test[["text", "unsafe"]]

In [3]:
train = train.loc[(train['unsafe'] >= 0.8) | (train['unsafe'] <= 0.2)]

In [4]:
def binary(prob):
    if prob < 0.5:
        return 0.
    else:
        return 1.0

In [5]:
train['unsafe'] = train['unsafe'].apply(binary)
test['unsafe'] = test['unsafe'].apply(binary)

In [6]:
train = train.sample(frac=1)

In [7]:
train, test

(                                                     text  unsafe
 44157   Стащил когда-то и б. Дайте соус, по пикче не г...     0.0
 128870  Каперсы всегда были отвратителтельны на вкус, ...     0.0
 130118  и че толку ждать, ну понравился человек сама/с...     0.0
 75764          Хорошо, что хронометриста не прислали ;)))     0.0
 21494   Вот, сидят в своей пещере и не знают, когда ин...     1.0
 ...                                                   ...     ...
 106842  Сокол это конечно показатель. Я нигде не говор...     0.0
 50636   Вот это подготовка! Девчонки - красотки! Хрупк...     0.0
 123155   fs, стало быть? Нормально, поиграть можно, да...     0.0
 126467              А что на счёт сроков давности штрафа?     0.0
 23715   да да, в однополые пары тоже пишут - "есть люб...     1.0
 
 [120120 rows x 2 columns],
                                                     text  unsafe
 0                       уровень ссачнее, чем ад в доом 3     1.0
 1      У нас несколько спортсмено

In [8]:
x_train = train['text'].tolist()
y_train = train['unsafe'].tolist()
x_test = test['text'].tolist()
y_test = test['unsafe'].tolist()

In [10]:
!pip install youtokentome

     |████████████████████████████████| 1.7 MB 866 kB/s eta 0:00:01


In [11]:
import youtokentome as yttm

In [22]:
yttm_train = '\n'.join(x_train)

In [21]:
vocab_size = 100000
train_path = '/kaggle/working/yttm_train.txt'
model_path = '/kaggle/working/pretrained_bpe.model'

In [23]:
with open(train_path, 'w') as f:
    f.write(yttm_train)

In [24]:
yttm.BPE.train(data=train_path, vocab_size=vocab_size, model=model_path)

In [25]:
tokenizer = yttm.BPE(model=model_path)

In [26]:
tokenizer.encode('Крокозябра')

[6353, 638, 1507, 1209]

In [27]:
tokenizer.decode([4,1])

['<UNK>']

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader

In [29]:
max_len = 45

In [30]:
class UnsafeData(Dataset):

    def __init__(self, texts, targets, tokenizer, max_len, pad_index = 0):
        
        super().__init__()
        
        self.texts = texts
        self.targets = targets        
        self.max_len = max_len
        self.pad_index = pad_index

    def __len__(self):
        
        return len(self.texts)
    
    def padding(self, text):
        
        text = text[:self.max_len]        
        text += [self.pad_index] * (self.max_len - len(text))        
        return text

    
    def __getitem__(self, index):
        x = self.texts[index]
        y = self.targets[index]
        
        x = tokenizer.encode(x)
        x = self.padding(x)
        
        x = torch.tensor(x).long()
        y = torch.tensor(y).float()
        
        return x, y

In [31]:
train_dataset = UnsafeData(x_train, y_train, tokenizer, max_len)
test_dataset = UnsafeData(x_test, y_test, tokenizer, max_len)

In [32]:
len(train_dataset), len(test_dataset)

(120120, 24501)

In [33]:
train_dataset[23000]

(tensor([  703,   955,   621,   969, 23864, 20989,  4478,  1019,  1793,   669,
          1023,  3498,   665,   877,  9734,   621, 30377,  2312, 58534,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]),
 tensor(1.))

In [34]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle = True)

In [35]:
for batch in test_loader:
    break
batch

[tensor([[  877,   783,   619,  ...,     0,     0,     0],
         [33467,  2351, 23926,  ...,     0,     0,     0],
         [ 1108,   705,   784,  ...,     0,     0,     0],
         ...,
         [ 5573,  3394,  4828,  ...,     0,     0,     0],
         [  799,  1292,   878,  ...,     0,     0,     0],
         [  724,   722,   703,  ...,     0,     0,     0]]),
 tensor([0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.])]

In [36]:
for x, y in test_loader:
    break

x.shape, y.shape

(torch.Size([32, 45]), torch.Size([32]))

In [26]:
y.type()

'torch.FloatTensor'

In [37]:
from torch import nn

In [38]:
class Classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        #packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [69]:
model = Classifier(
    vocab_size = 100000,
    embedding_dim = 400,
    hidden_dim = 256,
    output_dim = 1,
    n_layers = 6,
    bidirectional = True,
    dropout = 0.4
)

In [74]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCELoss()

In [41]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

In [53]:
def metrics(true, predictions):
    
    rounded_preds = torch.round(predictions)
    
    precision, recall, f1, _ = precision_recall_fscore_support(true, rounded_preds, average='binary', zero_division = 0)
    acc = accuracy_score(true, rounded_preds)
    #roc_auc = roc_auc_score(true, predictions)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
        #'roc_auc': roc_auc
    }
    
    

In [71]:
device = torch.device('cuda')
model.to(device)

Classifier(
  (embedding): Embedding(100000, 400)
  (lstm): LSTM(400, 256, num_layers=6, batch_first=True, dropout=0.4, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (act): Sigmoid()
)

In [44]:
from tqdm import tqdm

In [54]:
def train(model, loader, optimizer, criterion, last_n_losses=200, verbose=True):

    losses = []
    f_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    #roc_auc_scores = []

    progress_bar = tqdm(total=len(loader), disable=not verbose, desc='Train')

    model.train()

    for x, y in loader:

        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        yhat = model(x, 45).squeeze()
        
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        
        cur_metrics = metrics(y.cpu(), yhat.detach().cpu())


        losses.append(loss.item())
        f_scores.append(cur_metrics['f1'])
        accuracy_scores.append(cur_metrics['accuracy'])
        precision_scores.append(cur_metrics['precision'])
        recall_scores.append(cur_metrics['recall'])
        #roc_auc_scores.append(cur_metrics['roc_auc'])
        

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]), f1=np.mean(f_scores[-last_n_losses:]),
                                accuracy=np.mean(accuracy_scores[-last_n_losses:]))
        
        progress_bar.update()

    progress_bar.close()
    
    return {'loss': np.sum(losses)/len(loader), 'f_score': np.sum(f_scores)/len(loader), 'accuracy': np.sum(accuracy_scores)/len(loader),
           'precision': np.sum(precision_scores)/len(loader), 'recall': np.sum(recall_scores)/len(loader)}

In [46]:
import numpy as np

In [73]:
tqdm._instances.clear()

In [55]:
def evaluate(model, loader, criterion, last_n_losses=200, verbose=True):

    losses = []
    f_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    #roc_auc_scores = []

    progress_bar = tqdm(total=len(loader), disable=not verbose, desc='Eval')

    model.eval()
    with torch.no_grad():

        for x, y in loader:

            x = x.to(device)
            y = y.to(device)
        
            yhat = model(x, 45).squeeze()
        
            loss = criterion(yhat, y)
        
            cur_metrics = metrics(y.cpu(), yhat.detach().cpu())


            losses.append(loss.item())
            f_scores.append(cur_metrics['f1'])
            accuracy_scores.append(cur_metrics['accuracy'])
            precision_scores.append(cur_metrics['precision'])
            recall_scores.append(cur_metrics['recall'])
            #roc_auc_scores.append(cur_metrics['roc_auc'])
        

            progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]), f1=np.mean(f_scores[-last_n_losses:]),
                                accuracy=np.mean(accuracy_scores[-last_n_losses:]))
        
            progress_bar.update()

        progress_bar.close()
    
    return {'loss': np.sum(losses)/len(loader), 'f_score': np.sum(f_scores)/len(loader), 'accuracy': np.sum(accuracy_scores)/len(loader),
           'precision': np.sum(precision_scores)/len(loader), 'recall': np.sum(recall_scores)/len(loader)}

In [51]:
save_best_model_path = '/kaggle/working/best_model_state_dict.pth'
save_best_optimizer_path = '/kaggle/working/best_optimizer_state_dict.pth'

In [166]:
torch.save(model.state_dict(), save_best_model_path)

In [75]:
n_epochs = 7
best_valid_loss = float('inf')
patience = 0
for epoch in range(n_epochs):
     
    #train the model
    train_metrics = train(model, train_loader, optimizer, criterion)
    
    #evaluate the model
    valid_metrics = evaluate(model, test_loader, criterion)
    
    print(train_metrics)
    print(valid_metrics)
    
    #save the best model
    if valid_metrics['loss'] < best_valid_loss:
        best_valid_loss = valid_metrics['loss']
        torch.save(model.state_dict(), save_best_model_path)
        torch.save(optimizer.state_dict(), save_best_optimizer_path)
    else:
        patience +=1
        if patience>3:
            break
    
   

Eval: 100%|██████████| 766/766 [00:14<00:00, 52.15it/s, accuracy=0.696, f1=0, loss=0.611]


{'loss': 0.573664595511408, 'f_score': 0.021848227686293503, 'accuracy': 0.7218605709465459, 'precision': 0.07735276104349094, 'recall': 0.013952134997333844}
{'loss': 0.6046996908383955, 'f_score': 0.0, 'accuracy': 0.6988549825935597, 'precision': 0.0, 'recall': 0.0}


Eval: 100%|██████████| 766/766 [00:14<00:00, 52.72it/s, accuracy=0.741, f1=0.504, loss=0.538]


{'loss': 0.4682883356256757, 'f_score': 0.4968301171302024, 'accuracy': 0.7780367607884923, 'precision': 0.6341697316624061, 'recall': 0.44301352218155304}
{'loss': 0.5445108435872952, 'f_score': 0.48558470372112206, 'accuracy': 0.7324148327738407, 'precision': 0.5705544691021193, 'recall': 0.4449263554710106}


Train:   0%|          | 2/3754 [00:00<05:08, 12.15it/s, accuracy=0.953, f1=0.932, loss=0.217]

...

{'loss': 0.21885627167191224, 'f_score': 0.8466225085839, 'accuracy': 0.9200297460486593, 'precision': 0.8643222640047595, 'recall': 0.8462262964079603}
{'loss': 0.6933154677947881, 'f_score': 0.4927205643696674, 'accuracy': 0.7310258143727465, 'precision': 0.5698781020256216, 'recall': 0.45854003509888497}


Train:   0%|          | 2/3754 [00:00<05:04, 12.34it/s, accuracy=0.953, f1=0.885, loss=0.127]

{'loss': 0.13764838898119297, 'f_score': 0.9153161202759574, 'accuracy': 0.9560718566862014, 'precision': 0.9259462907305209, 'recall': 0.9148304610971884}
{'loss': 0.7770965414087704, 'f_score': 0.49839104859395406, 'accuracy': 0.7314104656222803, 'precision': 0.5632033559070496, 'recall': 0.4717250319519556}


Eval: 100%|██████████| 766/766 [00:14<00:00, 52.20it/s, accuracy=0.723, f1=0.504, loss=0.976]

{'loss': 0.09167837153859218, 'f_score': 0.9494041308321293, 'accuracy': 0.9736697522642515, 'precision': 0.9584049399105496, 'recall': 0.946450436565882}
{'loss': 0.9917727066471433, 'f_score': 0.5090480121844906, 'accuracy': 0.7200302281487008, 'precision': 0.5375705090869811, 'recall': 0.5102103941596723}
